In [58]:
pip install datasketch

In [67]:
from bs4 import BeautifulSoup
import os.path
import os
import time
import re
import codecs
import string
import binascii
import numpy as np
import random
from datasketch import MinHash, MinHashLSHForest


In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

!mkdir p1
!mkdir p2
!mkdir p3

!unzip /content/gdrive/MyDrive/colab_files/reut2.zip -d data

Mounted at /content/gdrive
Archive:  /content/gdrive/MyDrive/colab_files/reut2.zip
  inflating: data/reut2-000.sgm      
  inflating: data/reut2-001.sgm      
  inflating: data/reut2-002.sgm      
  inflating: data/reut2-003.sgm      
  inflating: data/reut2-004.sgm      
  inflating: data/reut2-005.sgm      
  inflating: data/reut2-006.sgm      
  inflating: data/reut2-007.sgm      
  inflating: data/reut2-008.sgm      
  inflating: data/reut2-009.sgm      
  inflating: data/reut2-010.sgm      
  inflating: data/reut2-011.sgm      
  inflating: data/reut2-012.sgm      
  inflating: data/reut2-013.sgm      
  inflating: data/reut2-014.sgm      
  inflating: data/reut2-015.sgm      
  inflating: data/reut2-016.sgm      
  inflating: data/reut2-017.sgm      
  inflating: data/reut2-018.sgm      
  inflating: data/reut2-019.sgm      
  inflating: data/reut2-020.sgm      
  inflating: data/reut2-021.sgm      


In [11]:
#show head and tail
def ht(List,Length):
  print(List[:Length])
  print(List[-Length:])

In [6]:
documents = []
selected_body = {}
data = ''

#open each file and combine contents

for file in os.listdir("data/"):
    if file.endswith(".sgm"):
        filename = os.path.join("data", file)
        f = codecs.open(filename, 'r', encoding='utf-8', errors='ignore')
        data = data + f.read()



In [8]:

#parse the html contents with BeautifulSoup

soup=BeautifulSoup(data, "html.parser")
all_b=soup.findAll('body')

In [12]:
i = 0
#obtain the articles in the content; outputs goes to 'documents'

for b in allb:
    selected_body[i] = b
    documents.append(re.sub(' +', ' ', str(b)\
             .replace("<body>", "")\
             .replace("</body>", "")\
             .translate(str.maketrans('', '', string.punctuation))\
             .replace("", "")\
             .replace("\n", " ").lower()))
    i += 1

#preview 'documents'
ht(documents,5)

['qtly div five cts vs five cts prior pay july 13 record june 30 reuter \x03', 'crude oil netback values in complex refineries rose sharply in europe and firmed in the us last friday from the previous week but fell sharply in singapore according to calculations by reuters pipeline the firmer tone to refining margins in europe and the us relected higher prices for petroleum products particularly gasoline and support from crude oil prices netback values for crude oil refined in northern europe rose substantially following strong gains in gasoline prices there brent is valued at 1945 dlrs up 56 cts a barrel or three pct from the previous week in the us gulf sweet crudes rose in value by 14 cts to 1933 dlrs for west texas intermediate up about 07 pct sour grades in the us gulf showed an increase of 33 cts a barrel for alaska north slope up 17 pct but netbacks for crude oil refined in singapore fell sharply down 15 cts to as much as 68 cts a barrel as ample distillate supplies weighed on pe

In [15]:
i = 0
d = {}

#get individual words in every doc
for value in documents:
    d[i] = value  
    d[i] = re.sub("[^\w]", " ", d[i]).split()  
    if d[i]: 
        i += 1
    else:
        del d[i]
        del b[i]


# (1) Given the Reuters-21578 dataset, 
# please calculate all k-shingles 
# and output the set representation of the text dataset as a matrix.

In [30]:
#get the value of k for shingles

while True:
    try:
        shingle_size = int(input("Please enter k value for k-shingles:")) #choose k=2
    except ValueError:
        print("Invalid input.")
        continue
    if shingle_size <= 0:
        continue
    else:
        break

Please enter k value for k-shingles:2


In [31]:
doc_names = []
total_shingles = 0
shingle_no = 0
shingle_sets = {}

all_shingles = set()

for i in range(0, len(d)): #iterate through all articless
    words = d[i] #words of current article
    doc_id = i #doc id

    doc_names.append(doc_id) 

    #sets that are cleared every doc
    shingles_in_word = set() 
    shingles_in_int = set()
    shingle = []


    for index in range(len(words) - shingle_size + 1): #obtain all shingles
        shingle = words[index:index + shingle_size] #current shingle
        shingle = ' '.join(shingle)        
        crc=binascii.crc32(str.encode(shingle)) & 0xffffffff     #encode shingle 

        if shingle not in all_shingles: 
            all_shingles.add(shingle) #the current shingle is not in list; add it
        if shingle not in shingles_in_word:  
            shingles_in_word.add(shingle)       
        if crc not in shingles_in_int:
            shingles_in_int.add(crc)
            shingle_no += 1
        else:
            del shingle
            index = index-1       
    shingle_sets[doc_id] = shingles_in_int
total_shingles = shingle_no

In [32]:
len(all_shingles)

671233

In [33]:
#get the value of k for shingles

while True:
    try:
        numd = int(input("Please enter number of documents: "))
    except ValueError:
        print("Invalid input.")
        continue
    if numd <= 0:
        continue
    else:
        break

matrix=[]
for i in all_shingles:
    tmp=[]
    for j in range(0, numd):
        if i in documents[j]:
            boolean=1
            tmp.append(boolean)
        else:
            boolean=0
            tmp.append(boolean)
    matrix.append(tmp)


Please enter number of documents: 10


In [35]:
with open("p1/" + str(shingle_size) + "_shingles.txt", 'w') as file:
    file.write("  ")
    for i in range(0, len(all_shingles)):
        for j in range(0, numd):
            file.write(str(matrix[i][j]))
        file.write("\n")

# (2) Given the set representation, compute the minhash signatures of all documents.

In [37]:
max_shingle_id = 2**32-1
next_prime = 4294967311
signatures = []

In [38]:
def pick_random(k):
    rand_list = []
    while k > 0:
        rand_index = random.randint(0,max_shingle_id) 
        while rand_index in rand_list:
            rand_index = random.randint(0,max_shingle_id) 
        rand_list.append(rand_index)
        k = k - 1  
    return rand_list

In [47]:
numHashes = len(shingle)
a = pick_random(numHashes)
b = pick_random(numHashes)
for doc_id in doc_names:
    shingle_id_set = shingle_sets[doc_id]
    signature = []
    for i in range(0, numHashes):
        minhash_code = next_prime+1
        for shingle_id in shingle_id_set:
            hash_code = (a[i] * shingle_id + b[i]) % next_prime 
            if hash_code < minhash_code:
                minhash_code = hash_code
        signature.append(minhash_code)
    signatures.append(signature)

In [45]:
with open("p2/minhash_signature.txt", 'w', newline = '') as file:
    file.write(str(signatures))

In [41]:
# (3) Implement the LSH algorithm and output the resulting candidate pairs of similar documents.

In [42]:
doc_no = 1000

In [43]:
def getTriangleIndex(i, j):
    if i == j:
        sys.stderr.write("cannot access triangle matrix")
        sys.exit(1)
    if j < i:
        temp = i
        i = j
        j = temp
    return int(i * (doc_no - (i + 1) / 2.0) + j - i) - 1

In [50]:
numElems = int(doc_no * (doc_no - 1) / 2)
JSim = [0 for x in range(numElems)]
estJSim = [0 for x in range(numElems)]
if doc_no <= 2500:
    for i in range(0, doc_no):
        if (i % 100) == 0:
            print(str(i) + " / " + str(doc_no))
        s1 = shingle_sets[doc_names[i]]
        for j in range(i + 1, doc_no):
            s2 = shingle_sets[doc_names[j]]
            JSim[getTriangleIndex(i, j)] = (len(s1.intersection(s2)) / len(s1.union(s2)))  
            
JSim

0 / 1000
100 / 1000
200 / 1000
300 / 1000
400 / 1000
500 / 1000
600 / 1000
700 / 1000
800 / 1000
900 / 1000


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0196078431372549,
 0.045454545454545456,
 0.015151515151515152,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.013333333333333334,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.03225806451612903,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.022222222222222223,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.006097560975609756,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0

In [69]:
def get_forest(data, perms):
    start_time = time.time()
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(shingle_sets):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [ ]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    m = MinHash(num_perm=perms)
    for s in all_shingles:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return resultb

In [71]:
get_forest(documents, 128)

TypeError: ignored